In [1]:
import cv2
import mediapipe as mp 
import joblib
import numpy as np 
import pyttsx3

In [2]:
string =''
out=''

In [5]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


# For webcam input
hands = mp_hands.Hands(max_num_hands=1,static_image_mode=False,min_detection_confidence=0.7)

cap = cv2.VideoCapture(0)

def data_clean(landmark):
    data = landmark[0]
    data = str(data)
    data = data.strip().split('\n')
   
    garbage = ['landmark {', '}']
    
    landmarks = []

    for i in data:
        if i not in garbage:
            landmarks.append(i)
            
    clean = []
    for i in landmarks:
        i = i.strip()
        clean.append(i[2:])
        
    finalClean = [ ]
    for i in range(0, len(clean)):
            if (i+1) % 3 != 0:
                finalClean.append(float(clean[i]))
            
    return([finalClean])
  

In [6]:
while cap.isOpened():
  success, image = cap.read()
  
  image = cv2.flip(image, 1)
  
  if not success:
    break

  # Flip the image horizontally for a later selfie-view display, and convert
  # the BGR image to RGB.
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # To improve performance, optionally mark the image as not writeable to
  # pass by reference.
  image.flags.writeable = False
  results = hands.process(image)
  # Draw the hand annotations on the image.
  image.flags.writeable = True
  
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

  if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
      mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cleaned_landmark = data_clean(results.multi_hand_landmarks)

    if cleaned_landmark:
      clf = joblib.load('RF_Model.pkl')
      y_pred = clf.predict(cleaned_landmark)
      image = cv2.putText(image, str(y_pred[0]), (22,150), cv2.FONT_HERSHEY_SIMPLEX,  2, (0,0,255), 2, cv2.LINE_AA) 
      out=str(y_pred[0])  
    
  font = cv2.FONT_HERSHEY_SIMPLEX  
  cv2.putText(image,"Word:", (22,34), font, 1, (0,255,0), 2, cv2.LINE_AA)
  cv2.putText(image,string, (22,64), font, 1, (0,255,0), 1, cv2.LINE_AA)
  cv2.putText(image,"S: Add  D: Delete  Esc: Quit", (22,470), font, 1, (255,100,100), 2, cv2.LINE_AA)  
  cv2.imshow('MediaPipe Hands', image) 
  
  
  key = cv2.waitKey(1)  
    
  if key== ord('s'):
    string = string+""+out 
    
  if key== ord('d'):
    string=string[:-1]  
        
  if key== 32:
    
    string = string+" ";      
        
  if key== 27:
    break

hands.close()
cap.release()
cv2.destroyAllWindows()

In [7]:
print(string)

In [8]:
engine = pyttsx3.init()
engine.say(string)
engine.runAndWait()